In [1]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

# Amdahl's Law

## The starting example -- sorting

In [2]:
render_code("./gpusort/main.cu", show="main")

// ./gpusort/main.cu:516-682 (167 lines)
int main( int argc, char** argv)
{ 

  // Create timers for each sort
    struct timeval time_start, time_end, program_start, program_end;
    double input_time=0;
    gettimeofday(&program_start, NULL);	

    sdkCreateTimer(&uploadTimer);
    sdkCreateTimer(&downloadTimer);
    sdkCreateTimer(&bucketTimer);
    sdkCreateTimer(&mergeTimer);
    sdkCreateTimer(&totalTimer);
    sdkCreateTimer(&cpuTimer);
	int numElements = 0;
	int method=0;
    // Number of elements in the test bed
    	gettimeofday(&time_start, NULL);
#ifdef READING_FROM_BINARY
        numElements = atoi(argv[2]);
	if (argc == 4)
		method = atoi(argv[3]);
#else
       	if(strcmp(argv[1],"r") ==0) {
	         numElements = SIZE; 
	    }
	else {
		FILE *fp;
	    fp = fopen(argv[1],"r");
	    if(fp == NULL) {
	        cout << "Error reading file" << endl;
	        exit(EXIT_FAILURE);
	    }
	    int count = 0;
	    float c;

	    while(fscanf(fp,"%f",&c) != EOF) {
	        count++;
        }
	    fclose(fp);
	    numElements = count;
    }
#endif

	cout << "Sorting list of " << numElements << " floats\n";
	// Generate random data
	// Memory space the list of random floats will take up
	int mem_size = numElements * sizeof(float); 
	// Allocate enough for the input list
	float *cpu_idata = (float *)malloc(mem_size);
	// Allocate enough for the output list on the cpu side
	float *cpu_odata = (float *)malloc(mem_size);
	// Allocate enough memory for the output list on the gpu side
	float *gpu_odata = (float *)malloc(mem_size);

	float datamin = FLT_MAX; 
	float datamax = -FLT_MAX; 
	if(strcmp(argv[1],"r")==0) {

	for (int i = 0; i < numElements; i++) {
	// Generate random floats between 0 and 1 for the input data
		cpu_idata[i] = ((float) rand() / RAND_MAX); 
	//Compare data at index to data minimum, if less than current minimum, set that element as new minimum
		datamin = min(cpu_idata[i], datamin);
	//Same as above but for maximum
		datamax = max(cpu_idata[i], datamax);
	}
    }	else {
	FILE *fp;
#ifdef READING_FROM_BINARY
	fp = fopen(argv[1],"rb");
	fread(cpu_idata,sizeof(float),numElements,fp);
	fclose(fp);
   	gettimeofday(&time_end, NULL);
    input_time = ((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0;
	printf("FileInput %lf seconds\n",input_time);
	
    /*	for(int i = 0; i < numElements; i++) {
	datamin = min(cpu_idata[i],datamin);
	datamax = max(cpu_idata[i],datamax);
	}*/

	datamin = 0.0;
	datamax = 16777215.0;
#else
	fp = fopen(argv[1],"r");
	for(int i = 0; i < numElements; i++) {
	fscanf(fp,"%f",&cpu_idata[i]);
	datamin = min(cpu_idata[i],datamin);
	datamax = max(cpu_idata[i],datamax);
	}
	fclose(fp);
   	gettimeofday(&time_end, NULL);
    input_time = ((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0;
	printf("FileInput %lf seconds\n",input_time);
    	gettimeofday(&time_start, NULL);	
#endif
	}
#ifndef CPU
	cout << "Sorting on GPU..." << flush; 
	// GPU Sort
	if(method == 0)
	for (int i = 0; i < TEST; i++) 
        MergeSort(cpu_idata, numElements, THREADS_PER_BLOCK, numElements/THREADS_PER_BLOCK);
//		gpu_quicksort(cpu_idata, gpu_odata, numElements);		
	else {
	// Using bitonic sort
	for (int i = 0; i < TEST; i++) 
		bitonic_sort(cpu_idata, gpu_odata, numElements);		
	}
#endif

#if defined(VERIFY) || (defined CPU)
	cout << "Sorting on CPU..." << flush; 
	// Start_CPU_Sort
	sdkStartTimer(&cpuTimer); 
	memcpy(cpu_odata, cpu_idata, mem_size);
    qsort(cpu_odata, numElements, sizeof(float), compare);
	sdkStopTimer(&cpuTimer); 
    // End_CPU_Sort 
	cout << "done.\n";
	#ifdef VERIFY
    cout << "Checking result..." << flush; 
	// Result checking
	int count = 0; 
	for(int i = 0; i < numElements; i++)
		if(fabs(cpu_odata[i] - gpu_odata[i])> 0.0002)
		{
			printf("Sort missmatch on element %d: \n", i); 
			printf("CPU = %f : GPU = %f\n", cpu_odata[i], gpu_odata[i]); 
			count++; 
			break; 
		}

## Where is the most time critical part of my program?

In [3]:
! cd gpusort; make clean; make
! perf record gpusort/hybridsort_cpu ~/courses/CSE142/demo/300000000.bin 134217728 2>> sort.csv

rm -f	*.o hybridsort hybridsort_cpu bitonic
/usr/local/cuda/bin/nvcc -DTIMER -O3 -w -g    -DCPU -DHAVE_LINUX_PERF_EVENT_H -DREADING_FROM_BINARY         main.cu -o hybridsort_cpu
/usr/local/cuda/bin/nvcc -DTIMER -O3 -w -g    -DHAVE_LINUX_PERF_EVENT_H -DREADING_FROM_BINARY         main.cu -o hybridsort
Sorting list of 134217728 floats
FileInput 1.828626 seconds
Sorting on CPU...done.
Total CPU execution time: 15.484981 seconds


In [4]:
! perf report --stdio > perf.out

Kernel address maps (/proc/{kallsyms,modules}) were restricted.

Check /proc/sys/kernel/kptr_restrict before running 'perf record'.

As no suitable kallsyms nor vmlinux was found, kernel samples
can't be resolved.

Samples in kernel modules can't be resolved as well.



In [4]:
! head -20 perf.out

# To display the perf.data header info, please use --header/--header-only options.
#
#
# Total Lost Samples: 0
#
# Samples: 78  of event 'cpu_atom/cycles/P'
# Event count (approx.): 74155853
#
# Overhead  Command         Shared Object         Symbol                  
# ........  ..............  ....................  ........................
#
    43.18%  hybridsort_cpu  [unknown]             [k] 0xffffffffa2299a9e
    23.13%  hybridsort_cpu  [unknown]             [k] 0xffffffffa2c1ae37
     2.75%  hybridsort_cpu  [unknown]             [k] 0xffffffffa1e50b3f
     2.64%  hybridsort_cpu  [unknown]             [k] 0xffffffffa2e01248
     2.09%  hybridsort_cpu  ld-linux-x86-64.so.2  [.] _dl_relocate_object
     1.37%  hybridsort_cpu  [unknown]             [k] 0xffffffffa1eefd03
     1.37%  hybridsort_cpu  [unknown]             [k] 0xffffffffa1ef1d90
     1.37%  hybridsort_cpu  [unknown]             [k] 0xffffffffa1df4eb9
     1.37%  hybridsort_cpu  [unknown]             [k] 0xffffffffa1e819

In [8]:
! objdump -D ./gpusort/hybridsort_cpu > hybridsort_cpu.dump

### Use gprof to figure out the timing breakdown

In [5]:
! cd gpusort; make clean; make EXTRA_FLAGS=-pg 
! cd ./gpusort; source ./run_CPU

rm -f	*.o hybridsort hybridsort_cpu bitonic
/usr/local/cuda/bin/nvcc -DTIMER -O3 -w -g  -pg  -DCPU -DHAVE_LINUX_PERF_EVENT_H -DREADING_FROM_BINARY         main.cu -o hybridsort_cpu
/usr/local/cuda/bin/nvcc -DTIMER -O3 -w -g  -pg  -DHAVE_LINUX_PERF_EVENT_H -DREADING_FROM_BINARY         main.cu -o hybridsort
Sorting list of 134217728 floats
FileInput 0.154118 seconds
Sorting on CPU...done.
Total CPU execution time: 26.180595 seconds
26.334753, 0.154118, 26.180595, 0.000000, 0.000000, 0.000000


In [6]:
! cd gpusort; gprof ./hybridsort_cpu ./gmon.out

Flat profile:

Each sample counts as 0.01 seconds.
  %   cumulative   self              self     total           
 time   seconds   seconds    calls  Ts/call  Ts/call  name    
100.00      1.78     1.78                             compare(void const*, void const*)
  0.00      1.78     0.00        6     0.00     0.00  __cudaUnregisterBinaryUtil()
  0.00      1.78     0.00        6     0.00     0.00  StopWatchLinux::~StopWatchLinux()
  0.00      1.78     0.00        5     0.00     0.00  StopWatchLinux::getTime()
  0.00      1.78     0.00        1     0.00     0.00  StopWatchLinux::stop()
  0.00      1.78     0.00        1     0.00     0.00  StopWatchLinux::start()

 %         the percentage of the total running time of the
time       program used by this function.

cumulative a running sum of the number of seconds accounted
 seconds   for by this function and those listed above it.

 self      the number of seconds accounted for by this
seconds    function alone.  This is the major sort 